In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
import os
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
from flask import Flask, render_template
from pprint import pprint

In [3]:
#Reading our first Kaggle data set into a dataframe using pandas.read_csv + capitalizing the T in our 'Title' column for merging with the next data sets.  

csvpath = "Resources/netflix_titles.csv"
netflix_title_df= pd.read_csv(csvpath)
netflix_title_df = netflix_title_df.rename(columns={'title': 'Title'})
netflix_title_df.head()


,show_id,type,Title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [4]:
# Setting up our wikipedia url for webscraping  

url = 'https://en.wikipedia.org/wiki/List_of_Netflix_original_programming'

In [6]:
# Using pandas to read the wikipedia html on the Netflix Original Programming page, assigned this html to a variable called tables.  

tables = pd.read_html(url)
tables

[                         Title                               Genre  \
 0              Stranger Things              Science fiction/horror   
 1                    The Crown                    Historical drama   
 2                        Ozark                         Crime drama   
 3                Lost in Space                     Science fiction   
 4               Narcos: Mexico                         Crime drama   
 5         The Umbrella Academy                    Superhero action   
 6                 Black Summer                        Zombie drama   
 7                 Another Life               Science fiction drama   
 8                 Criminal: UK  Police procedural anthology series   
 9                 Raising Dion     Superhero/Science fiction drama   
 10                Virgin River                      Romantic drama   
 11                 The Witcher                       Fantasy drama   
 12                 Locke & Key                   Horror teen drama   
 13   

In [5]:
# The wikipedia page had its Netflix original titles organized into tables by genre, so we assigned each genre to a specific data frame.

# df_dramas = tables[0]
# df_comedies = tables[1]
# df_adultanim = tables[2]
# df_anime = tables[3]
# df_arb = tables[5]
# df_danish = tables[6]
# df_french = tables[7]
# df_german = tables[8]
# df_hindi = tables[9]
# df_italian = tables[10]
# df_japanese = tables[11]
# df_korean= tables[12]
# df_nor = tables[13]
# df_polish = tables[14]
# df_port = tables[15]
# df_spanish = tables[16]
# df_swedish = tables[17]
# df_tamil = tables[18]
# df_turkish = tables[19]
# df_other = tables[20]
# df_docu = tables[21]
# df_reality = tables[22]

In [6]:
# After assigning each genre to a variable we used a for loop to merge them into one final data frame and deleted some extraneous columns.  

for i in range(23):
    if i==0:
        df_final = pd.merge(tables[i], tables[i+1], how='outer')
    elif i !=4 & i != 1:
        df_final = pd.merge(df_final,tables[i], how='outer')
del df_final['Language']
del df_final['Unnamed: 6']
del df_final['Unnamed: 7']
df_final.head()

,Title,Genre,Premiere,Seasons,Runtime,Status
0,Stranger Things,Science fiction/horror,"July 15, 2016","3 seasons, 25 episodes",42–78 min.,Season 4 due to premiere in 2022[1]
1,The Crown,Historical drama,"November 4, 2016","4 seasons, 40 episodes",47–61 min.,Renewed for seasons 5 and 6 (final)[2][3]
2,Ozark,Crime drama,"July 21, 2017","3 seasons, 30 episodes",52–80 min.,Renewed for two part final season[4]
3,Lost in Space,Science fiction,"April 13, 2018","2 seasons, 20 episodes",39–66 min.,Renewed for final season[5]
4,Narcos: Mexico,Crime drama,"November 16, 2018","2 seasons, 20 episodes",45–70 min.,Renewed[6]


In [7]:
# Next we merged our kaggle data set with the newly created wikipedia data frame. 

netflix_title_df
wiki_kaggle_df = pd.merge(netflix_title_df,df_final, how='inner', on='Title')
wiki_kaggle_df.head()

,show_id,type,Title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Seasons,Runtime,Status
0,s219,TV Show,A Queen Is Born,Carla Barros,"Gloria Groove, Alexia Twister",Brazil,"November 11, 2020",2020,TV-14,1 Season,"International TV Shows, Reality TV",Gloria Groove and Alexia Twister make drag dre...,Reality,"November 11, 2020","1 season, 6 episodes",42–44 min.,Pending
1,s325,TV Show,After Life,NaN,"Ricky Gervais, Tom Basden, Tony Way, Diane Mor...",United Kingdom,"April 24, 2020",2020,TV-MA,2 Seasons,"British TV Shows, International TV Shows, TV C...",Struggling to come to terms with his wife's de...,Comedy,"March 8, 2019","2 seasons, 12 episodes",25–31 min.,Renewed[41]
2,s342,TV Show,Aggretsuko,NaN,"Kaolip, Komegumi Koiwasaki, Maki Tsuruta, Soht...",Japan,"August 27, 2020",2020,TV-14,3 Seasons,"Anime Series, International TV Shows","Frustrated with her thankless office job, Rets...",Workplace comedy,"April 20, 2018","3 seasons, 30 episodes",15–23 min.,Renewed[59]
3,s394,TV Show,Alice in Borderland,NaN,"Kento Yamazaki, Tao Tsuchiya, Nijiro Murakami,...",Japan,"December 10, 2020",2020,TV-MA,1 Season,"International TV Shows, TV Action & Adventure,...",An aimless gamer and his two friends find them...,Fantasy,"December 10, 2020","1 season, 8 episodes",41–52 min.,Renewed[98]
4,s399,TV Show,Alien Worlds,NaN,NaN,United Kingdom,"December 2, 2020",2020,TV-PG,1 Season,"British TV Shows, Docuseries, International TV...",Applying the laws of life on Earth to the rest...,Docu-series,"December 2, 2020","1 season, 4 episodes",41–47 min.,Pending


In [8]:
# Next step is to clean up the wiki_kaggle_df to prepare it for the final stage of the webscraping process.  
# We created a new column named "Title URL" which is the netflix title with underscores instead of spaces to use in our scraping of Rotten Tomatoes.  
# We also changed the values in the type column, "TV Show" and "Movie", to "tv" and "m" as that is how they are designated on Rotten Tomatoes.  

wiki_kaggle_df['Title URL'] = wiki_kaggle_df['Title'].str.replace(' ','_')
wiki_kaggle_df['type'].replace('TV Show','tv', inplace=True)
wiki_kaggle_df['type'].replace('Movie','m', inplace=True)
wiki_kaggle_df.head()

,show_id,type,Title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Seasons,Runtime,Status,Title URL
0,s219,tv,A Queen Is Born,Carla Barros,"Gloria Groove, Alexia Twister",Brazil,"November 11, 2020",2020,TV-14,1 Season,"International TV Shows, Reality TV",Gloria Groove and Alexia Twister make drag dre...,Reality,"November 11, 2020","1 season, 6 episodes",42–44 min.,Pending,A_Queen_Is_Born
1,s325,tv,After Life,NaN,"Ricky Gervais, Tom Basden, Tony Way, Diane Mor...",United Kingdom,"April 24, 2020",2020,TV-MA,2 Seasons,"British TV Shows, International TV Shows, TV C...",Struggling to come to terms with his wife's de...,Comedy,"March 8, 2019","2 seasons, 12 episodes",25–31 min.,Renewed[41],After_Life
2,s342,tv,Aggretsuko,NaN,"Kaolip, Komegumi Koiwasaki, Maki Tsuruta, Soht...",Japan,"August 27, 2020",2020,TV-14,3 Seasons,"Anime Series, International TV Shows","Frustrated with her thankless office job, Rets...",Workplace comedy,"April 20, 2018","3 seasons, 30 episodes",15–23 min.,Renewed[59],Aggretsuko
3,s394,tv,Alice in Borderland,NaN,"Kento Yamazaki, Tao Tsuchiya, Nijiro Murakami,...",Japan,"December 10, 2020",2020,TV-MA,1 Season,"International TV Shows, TV Action & Adventure,...",An aimless gamer and his two friends find them...,Fantasy,"December 10, 2020","1 season, 8 episodes",41–52 min.,Renewed[98],Alice_in_Borderland
4,s399,tv,Alien Worlds,NaN,NaN,United Kingdom,"December 2, 2020",2020,TV-PG,1 Season,"British TV Shows, Docuseries, International TV...",Applying the laws of life on Earth to the rest...,Docu-series,"December 2, 2020","1 season, 4 episodes",41–47 min.,Pending,Alien_Worlds


In [9]:
# Next we create an empty list called movie_url to hold the results of our for loop constructing the rotten tomatoes links for each netflix tv show and movie.  

movie_url = []
base_url = 'https://www.rottentomatoes.com/'
for i in range(len(wiki_kaggle_df['Title URL'])):
    movie_url.append(base_url + wiki_kaggle_df['type'][i] + '/' + wiki_kaggle_df['Title URL'][i])
    print(movie_url[i])

https://www.rottentomatoes.com/tv/A_Queen_Is_Born
https://www.rottentomatoes.com/tv/After_Life
https://www.rottentomatoes.com/tv/Aggretsuko
https://www.rottentomatoes.com/tv/Alice_in_Borderland
https://www.rottentomatoes.com/tv/Alien_Worlds
https://www.rottentomatoes.com/tv/Almost_Happy
https://www.rottentomatoes.com/tv/Anitta:_Made_In_Honório
https://www.rottentomatoes.com/tv/Another_Life
https://www.rottentomatoes.com/tv/Aunty_Donna's_Big_Ol'_House_of_Fun
https://www.rottentomatoes.com/tv/B:_The_Beginning
https://www.rottentomatoes.com/tv/Bad_Boy_Billionaires:_India
https://www.rottentomatoes.com/tv/Barbarians
https://www.rottentomatoes.com/tv/Best_Leftovers_Ever!
https://www.rottentomatoes.com/tv/Bhaag_Beanie_Bhaag
https://www.rottentomatoes.com/tv/Big_Mouth
https://www.rottentomatoes.com/tv/Biohackers
https://www.rottentomatoes.com/tv/Black_Summer
https://www.rottentomatoes.com/tv/Bling_Empire
https://www.rottentomatoes.com/tv/Blood_&_Water
https://www.rottentomatoes.com/tv/Blood_o

In [10]:
# Now we set up a loop to go visit our movie urls and use beautifulsoup to parse their HTML for the span and class 
# holding the rotten tomatoes Audience and Critic ratings and then saved those scores into a list.

ratings_list = []

for i in movie_url:        
    movie_results = requests.get(i)
    movie_soup = BeautifulSoup(movie_results.text, 'html.parser')
    print(i)
    try:
        ratings = movie_soup.find_all('span', class_='mop-ratings-wrap__percentage')
        ratings_list.append(ratings)
        print('---------')
    except:
        ratings_list.append('NA')
        print('No ratings available.')
        print('---------')
print(ratings_list)

https://www.rottentomatoes.com/tv/A_Queen_Is_Born
---------
https://www.rottentomatoes.com/tv/After_Life
---------
https://www.rottentomatoes.com/tv/Aggretsuko
---------
https://www.rottentomatoes.com/tv/Alice_in_Borderland
---------
https://www.rottentomatoes.com/tv/Alien_Worlds
---------
https://www.rottentomatoes.com/tv/Almost_Happy
---------
https://www.rottentomatoes.com/tv/Anitta:_Made_In_Honório
---------
https://www.rottentomatoes.com/tv/Another_Life
---------
https://www.rottentomatoes.com/tv/Aunty_Donna's_Big_Ol'_House_of_Fun
---------
https://www.rottentomatoes.com/tv/B:_The_Beginning
---------
https://www.rottentomatoes.com/tv/Bad_Boy_Billionaires:_India
---------
https://www.rottentomatoes.com/tv/Barbarians
---------
https://www.rottentomatoes.com/tv/Best_Leftovers_Ever!
---------
https://www.rottentomatoes.com/tv/Bhaag_Beanie_Bhaag
---------
https://www.rottentomatoes.com/tv/Big_Mouth
---------
https://www.rottentomatoes.com/tv/Biohackers
---------
https://www.rottentomat

In [11]:
# Here we separate our audience and critic scores into their own lists using a for loop
# we were sure to include all NA's which are Netflix titles that do not have enough reviews to receive RT scores.
# We also used .string to convert list values into a navigable string for future usage

t_ratings = []
a_ratings = []
for i in range(0,138):
    try:
        t_ratings.append(ratings_list[i][0].string)
    except:
        t_ratings.append('NA')
    try:
        a_ratings.append(ratings_list[i][1].string)
    except:
        a_ratings.append('NA')

type(t_ratings[1])

bs4.element.NavigableString

In [ ]:
len(T_ratings)

In [13]:
# We had mismatches in the length of our two ratings lists which turned out to be due to the way NAs were being assigned
# To fix this we created new ratings lists 

T_ratings = []
A_ratings = []
for i in t_ratings:
    if i == 'NA':
        T_ratings.append(i)
        print(i)
    else:
        number = filter(str.isdigit, i)
        number2 = "".join(number)
        T_ratings.append(number2)
        print(number2)

for i in a_ratings:
    if i == 'NA':
        A_ratings.append(i)
        print(i)
    else:
        number = filter(str.isdigit, i)
        number2 = "".join(number)
        A_ratings.append(number2)
        print(number2)

NA
75
100
83
100
NA
NA
6
NA
NA
NA
86
NA
63
99
100
89
90
NA
100
88
80
NA
67
NA
NA
60
89
88
95
94
65
92
71
NA
14
97
63
100
NA
90
NA
100
NA
67
NA
92
86
NA
NA
88
38
100
70
NA
NA
NA
NA
98
63
88
NA
NA
NA
NA
76
NA
NA
49
NA
NA
97
NA
93
NA
NA
NA
NA
NA
95
95
78
81
86
51
96
57
40
NA
70
83
61
NA
97
57
NA
NA
NA
NA
57
55
77
89
38
93
NA
NA
80
78
100
NA
NA
NA
93
50
90
NA
60
NA
100
73
NA
NA
NA
83
NA
NA
68
53
37
73
60
80
NA
68
80
62
NA
NA
90
92
94
63
NA
NA
59
NA
NA
NA
79
NA
NA
76
71
62
52
NA
81
72
NA
NA
80
NA
NA
93
89
NA
49
93
71
NA
76
NA
NA
74
55
NA
NA
NA
NA
NA
NA
24
NA
75
11
NA
NA
89
NA
NA
57
NA
NA
NA
NA
72
50
76
NA
NA
NA
NA
79
NA
NA
NA
NA
NA
77
NA
71
NA
NA
NA
NA
NA
88
91
79
89
NA
NA
NA
NA
52
NA
88
72
69
NA
86
NA
NA
NA
NA
NA
71
67
NA
90
75
91
NA
NA
61
73
88
NA
NA
NA
NA
NA
91
NA
NA
NA
74
NA
NA
NA
NA
87
NA
NA
91
51
30
NA
NA
56
NA
74
91
71
NA


In [14]:
wiki_kaggle_df['Tomatometer Score'] = T_ratings
wiki_kaggle_df['Audience Score'] = A_ratings
wiki_kaggle_df.head()

,show_id,type,Title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Seasons,Runtime,Status,Title URL,Tomatometer Score,Audience Score
0,s219,tv,A Queen Is Born,Carla Barros,"Gloria Groove, Alexia Twister",Brazil,"November 11, 2020",2020,TV-14,1 Season,"International TV Shows, Reality TV",Gloria Groove and Alexia Twister make drag dre...,Reality,"November 11, 2020","1 season, 6 episodes",42–44 min.,Pending,A_Queen_Is_Born,NA,NA
1,s325,tv,After Life,NaN,"Ricky Gervais, Tom Basden, Tony Way, Diane Mor...",United Kingdom,"April 24, 2020",2020,TV-MA,2 Seasons,"British TV Shows, International TV Shows, TV C...",Struggling to come to terms with his wife's de...,Comedy,"March 8, 2019","2 seasons, 12 episodes",25–31 min.,Renewed[41],After_Life,75,90
2,s342,tv,Aggretsuko,NaN,"Kaolip, Komegumi Koiwasaki, Maki Tsuruta, Soht...",Japan,"August 27, 2020",2020,TV-14,3 Seasons,"Anime Series, International TV Shows","Frustrated with her thankless office job, Rets...",Workplace comedy,"April 20, 2018","3 seasons, 30 episodes",15–23 min.,Renewed[59],Aggretsuko,100,92
3,s394,tv,Alice in Borderland,NaN,"Kento Yamazaki, Tao Tsuchiya, Nijiro Murakami,...",Japan,"December 10, 2020",2020,TV-MA,1 Season,"International TV Shows, TV Action & Adventure,...",An aimless gamer and his two friends find them...,Fantasy,"December 10, 2020","1 season, 8 episodes",41–52 min.,Renewed[98],Alice_in_Borderland,83,94
4,s399,tv,Alien Worlds,NaN,NaN,United Kingdom,"December 2, 2020",2020,TV-PG,1 Season,"British TV Shows, Docuseries, International TV...",Applying the laws of life on Earth to the rest...,Docu-series,"December 2, 2020","1 season, 4 episodes",41–47 min.,Pending,Alien_Worlds,100,63


In [15]:

final_df = wiki_kaggle_df

In [16]:
# Removing unneeded columns (show_id, listed_in, description)

del final_df['show_id']
del final_df['listed_in']
del final_df['description']
del final_df['Title URL']
final_df.head()

,type,Title,director,cast,country,date_added,release_year,rating,duration,Genre,Premiere,Seasons,Runtime,Status,Tomatometer Score,Audience Score
0,tv,A Queen Is Born,Carla Barros,"Gloria Groove, Alexia Twister",Brazil,"November 11, 2020",2020,TV-14,1 Season,Reality,"November 11, 2020","1 season, 6 episodes",42–44 min.,Pending,NA,NA
1,tv,After Life,NaN,"Ricky Gervais, Tom Basden, Tony Way, Diane Mor...",United Kingdom,"April 24, 2020",2020,TV-MA,2 Seasons,Comedy,"March 8, 2019","2 seasons, 12 episodes",25–31 min.,Renewed[41],75,90
2,tv,Aggretsuko,NaN,"Kaolip, Komegumi Koiwasaki, Maki Tsuruta, Soht...",Japan,"August 27, 2020",2020,TV-14,3 Seasons,Workplace comedy,"April 20, 2018","3 seasons, 30 episodes",15–23 min.,Renewed[59],100,92
3,tv,Alice in Borderland,NaN,"Kento Yamazaki, Tao Tsuchiya, Nijiro Murakami,...",Japan,"December 10, 2020",2020,TV-MA,1 Season,Fantasy,"December 10, 2020","1 season, 8 episodes",41–52 min.,Renewed[98],83,94
4,tv,Alien Worlds,NaN,NaN,United Kingdom,"December 2, 2020",2020,TV-PG,1 Season,Docu-series,"December 2, 2020","1 season, 4 episodes",41–47 min.,Pending,100,63


In [1]:
# In order to load data into MongoDB we had to convert the values into dictionary records and reset the index.

data.reset_index(inplace=True)
data_dict = data.to_dict("records")
company.insert_one({"index":"Sensex","data":data_dict})

In [18]:
Netflix_json = final_df.to_json(orient='index')
type(Netflix_json)

str

In [19]:
final_df.reset_index(inplace=True)
Netflix_dict = final_df.to_dict(orient="list")
Netflix_dict

{'index': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137],
 'type': ['tv',
  'tv',
  'tv',
  'tv',
  'tv',
  'tv',
  'tv',
  'tv',
  'tv',
  'tv',
  'tv',
  'tv',
  'tv',
  'tv',
  'tv',
  'tv'

In [20]:
# Setting up our flask instance and connection to mongo database.  

app = Flask(__name__)
conn = 'mongodb://localhost:27017'

In [21]:
client = pymongo.MongoClient(conn)

In [22]:
db = client.Netflix_db

In [23]:
# Inserting data into MongoDB under the name Netflix_Dict

db.scores.insert_many([Netflix_dict])

In [24]:
# Checking the load was done correctly

test = db.scores.find()
for i in test:
    pprint(i)

{'Audience Score': ['NA',
                    '90',
                    '92',
                    '94',
                    '63',
                    'NA',
                    'NA',
                    '59',
                    'NA',
                    'NA',
                    'NA',
                    '79',
                    'NA',
                    'NA',
                    '76',
                    '71',
                    '62',
                    '52',
                    'NA',
                    '81',
                    '72',
                    'NA',
                    'NA',
                    '80',
                    'NA',
                    'NA',
                    '93',
                    '89',
                    'NA',
                    '49',
                    '93',
                    '71',
                    'NA',
                    '76',
                    'NA',
                    'NA',
                    '74',
                    '55',
            